In [126]:
import pandas as pd
from scipy.stats import mstats
import folium
import numpy as np

readings_df = pd.read_csv('https://docs.google.com/spreadsheets/d/1WuwQwoloZsbl9vJoMUj7COMhiPVo8SncIU85dG_vjow/export?gid=0&format=csv',
                          skiprows=[0,1,2])
location_df = pd.read_csv('https://docs.google.com/spreadsheets/d/1WuwQwoloZsbl9vJoMUj7COMhiPVo8SncIU85dG_vjow/export?gid=1752938175&format=csv')
location_df['station_id'] = location_df['LARWMP Station ID'].apply(lambda x: int(x[-3:]))
m = folium.Map(location=[34.0407, -118.2468],
                 zoom_start=12,
                 tiles='Stamen Terrain')

In [87]:
SINGLE_SAMPLE_LIMT = 235
GEO_MEAN_LIMIT = 126
SAMPLE_RANGE = 35
readings_df['date'] = pd.to_datetime(readings_df['DATE'])

In [88]:
cur_date = datetime.date.today()
mask = (readings_df['date'] > cur_date - datetime.timedelta(days=SAMPLE_RANGE)) \
        & (readings_df['date'] <= cur_date)

In [89]:
readings = readings_df.loc[mask]


In [90]:
readings = readings.set_index('date')
del(readings['DATE'])

In [92]:
readings.replace(to_replace='<10', value=5, inplace=True)
readings.replace(to_replace='NS', value=np.nan)


,200,203,204,206,208,209,210,213,214,215,216,217,218,219
date,,,,,,,,,,,,,,
2017-07-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,41,41,NaN,NaN
2017-07-15,97.0,10,350,63.0,145,NaN,52,41,332.0,NaN,NaN,NaN,238.0,63.0
2017-07-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,262,156,41,NaN,NaN
2017-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,336,160,63,NaN,NaN
2017-07-30,51.0,10,145,52.0,118,NaN,110,20,583.0,NaN,NaN,NaN,85.0,265.0
2017-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51,146,41,NaN,NaN
2017-08-06,620.0,5,86,10.0,120,NaN,20,5,1090.0,NaN,NaN,NaN,272.0,231.0
2017-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1010,1260,1320,NaN,NaN


In [123]:
output_color_data = {}
def set_color(c):
    num_readings = c.count()
    if num_readings >= 5:
        c = c.dropna()
        c = pd.to_numeric(c)
        geo_mean = sp.stats.mstats.gmean(c.values)
        if geo_mean <= 127 or (c > 235).count() <= 2: 
            """Green – Rolling Geo Mean using the last 5 sample results within 35 days is 
            <127 and <2 single sample results >235 using those same 5 sample results."""
            return 'green'
        elif geo_mean >= 126 and geo_mean <= 160 or (c > 235).count() > 235:
            """Rolling Geo Mean is >126 and <160 and 2 or more single samples >235"""
            return 'yellow'
        else:
            return 'red'
    else:
        return 'gray'
    

In [125]:
colors = readings.apply(set_color)

In [127]:
loca